# Optimizer
## Proyecto final -- Optimización y Simulación
### Alejandro de Haro, Jose María Benítez, Daniel Escobosa, Pablo Berástegui

In [39]:
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
import numpy as np

In [40]:
df = pd.read_csv('../data/players_fifa18_sample.csv')

In [41]:
chemistry = pd.read_csv('../data/sample_chemistry_edges_fifa18.csv')

In [42]:
chemistry.head()

,player_id_i,player_id_j,chem
0,222952,235051,0
1,193064,222952,0
2,143603,222952,0
3,222952,233616,0
4,163907,222952,0


In [43]:
model = pe.ConcreteModel()

### Sets

In [44]:
# usar player_id como conjunto para que coincida con el CSV de química
relevant_cols = ['player_id', 'short_name', 'overall', 'value_eur', 'club_team_id','club_name', 'position', 'league_id', 'league_name', 'nationality_id','nationality_name', 'pace','shooting','passing','dribbling','defending','physic']

model.player = pe.Set(initialize=df['player_id'].astype(int).tolist())
model.club = pe.Set(initialize=df['club_team_id'].unique().tolist())
model.position = pe.Set(initialize=df['position'].unique().tolist())
model.nationality = pe.Set(initialize=df['nationality_id'].unique().tolist())
model.league = pe.Set(initialize=df['league_id'].unique().tolist())

### Parameters

In [45]:
# overall como dict por player_id
overall_dict = df.set_index('player_id')['overall'].astype(float).to_dict()
model.overall = pe.Param(model.player, initialize=overall_dict, within=pe.NonNegativeReals)

pace_dict = df.set_index('player_id')['pace'].astype(float).to_dict()
model.pace = pe.Param(model.player, initialize=pace_dict, within=pe.NonNegativeReals)

value_dict = df.set_index('player_id')['value_eur'].astype(float).to_dict()
model.value= pe.Param(model.player, initialize=value_dict, within=pe.NonNegativeReals)

shooting_dict = df.set_index('player_id')['shooting'].astype(float).to_dict()
model.shooting = pe.Param(model.player, initialize=shooting_dict, within=pe.NonNegativeReals)

passing_dict = df.set_index('player_id')['passing'].astype(float).to_dict()
model.passing = pe.Param(model.player, initialize=passing_dict, within=pe.NonNegativeReals)

dribbling_dict = df.set_index('player_id')['dribbling'].astype(float).to_dict()
model.dribbling = pe.Param(model.player, initialize=dribbling_dict, within=pe.NonNegativeReals)

defending_dict = df.set_index('player_id')['defending'].astype(float).to_dict()
model.defending = pe.Param(model.player, initialize=defending_dict, within=pe.NonNegativeReals)

physic_dict = df.set_index('player_id')['physic'].astype(float).to_dict()
model.physic = pe.Param(model.player, initialize=physic_dict, within=pe.NonNegativeReals)

In [46]:
# construir dict de química (asegurar int)
chemistry_dict = { (int(r.player_id_i), int(r.player_id_j)): float(r.chem)
                   for _, r in chemistry.iterrows() }

# si quieres que la química sea simétrica (chem[i,j] == chem[j,i]) descomenta:
chem_sym = {}
for (i,j), w in chemistry_dict.items():
    chem_sym[(i,j)] = w
    chem_sym[(j,i)] = w
chemistry_dict = chem_sym

model.chemistry = pe.Param(model.player, model.player,
                           initialize=chemistry_dict,
                           default=0.0,
                           within=pe.NonNegativeReals)

Conexiones de la formación 4-3-3.   SET

In [47]:
                      #  P LB LCB RCB RB LCM CM RCM LW ST RW
conections = np.array([ [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
                        [1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                        [1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
                        [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0],
                        [0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0],
                        [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
                        [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
                        [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
                        [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
])

Comprobación de simetría

In [48]:
conections.T == conections

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  Tr

In [49]:
connections = np.array([
    [0,0,1,1,0,0,0,0,0,0,0],
    [0,0,1,0,0,1,0,0,0,0,0],
    [1,1,0,1,0,0,1,0,0,0,0],
    [1,0,1,0,1,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,1,0,0,0],
    [0,1,0,0,0,0,1,0,1,0,0],
    [0,0,1,1,0,1,0,1,0,1,0],
    [0,0,0,0,1,0,1,0,0,0,1],
    [0,0,0,0,0,1,0,0,0,1,0],
    [0,0,0,0,0,0,1,0,1,0,1],
    [0,0,0,0,0,0,0,1,0,1,0],
], dtype=int)

POSITIONS = ["GK","LB","CI","CD","RB","MI","CM","MD","LW","ST","RW"]
# Mapa posición -> índice
pos_idx = {p:i for i,p in enumerate(POSITIONS)}

def conn_init(m, i, j):
    return int(connections[pos_idx[i], pos_idx[j]])

model.conn = pe.Param(
    model.position, model.position,
    initialize=conn_init,
    within=pe.Binary,
    default=0
)

In [53]:
raw_pos = df.set_index('player_id')['position'].to_dict()

can_play = {}
for pid, rpos in raw_pos.items():
    pid = int(pid)

    if rpos == "CB":
        for pos in ["CI", "CD"]:
            can_play[(pid, pos)] = 1
    elif rpos == "CM":
        for pos in ["MI", "CM", "MD"]:
            can_play[(pid, pos)] = 1
    else:
        # si el CSV ya trae la misma etiqueta que la formación
        if rpos in POSITIONS:
            can_play[(pid, rpos)] = 1

model.can_play = pe.Param(
    model.player, model.position,
    initialize=can_play,
    within=pe.Binary,
    default=0
)

## Variables

In [61]:
model.x = pe.Var(model.player, model.position, within=pe.Binary)

# 1 si el jugador p está seleccionado (en alguna posición)
model.sel = pe.Var(model.player, within=pe.Binary)

def link_sel_rule(m, p):
    return m.sel[p] == sum(m.x[p, pos] for pos in m.position)
model.link_sel = pe.Constraint(model.player, rule=link_sel_rule)

'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


## Funcion objetivo

In [62]:
def obj_rule(model):
    return sum(model.overall[p]*model.x[p,pos] for p in model.player for pos in model.position)
model.total_cost = pe.Objective(rule=obj_rule, sense=pe.maximize)

'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.AbstractScalarObjective'>).
This is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


## Constraints

In [63]:
def position_availability_rule(m, p, pos):
    return m.x[p, pos] <= m.can_play[p, pos]

model.position_availability = pe.Constraint(model.player, model.position, rule=position_availability_rule)

(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [64]:
# Un jugador como máximo en una posición
def atmost_one_pos_rule(m, p):
    return sum(m.x[p, pos] for pos in m.position) <= 1
model.atmost_one_pos = pe.Constraint(model.player, rule=atmost_one_pos_rule)

# Cada posición de la formación se cubre exactamente con 1 jugador
def exactly_one_player_per_pos_rule(m, pos):
    return sum(m.x[p, pos] for p in m.player) == 1
model.one_player_per_pos = pe.Constraint(model.position, rule=exactly_one_player_per_pos_rule)

# (Opcional) tamaño del equipo (redundante si cubres 11 posiciones)
model.team_size = pe.Constraint(expr=sum(model.sel[p] for p in model.player) == 11)

# Restricción de "puede jugar en la posición"
def position_availability_rule(m, p, pos):
    return m.x[p, pos] <= m.can_play[p, pos]
model.position_availability = pe.Constraint(model.player, model.position, rule=position_availability_rule)

(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.In

In [65]:
solver = po.SolverFactory('gurobi')
solver.solve(model, tee=False)

{'Problem': [{'Name': 'x1', 'Lower bound': 904.0, 'Upper bound': 904.0, 'Number of objectives': 1, 'Number of constraints': 7162, 'Number of variables': 6600, 'Number of binary variables': 6600, 'Number of integer variables': 6600, 'Number of continuous variables': 0, 'Number of nonzeros': 25300, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.011591911315917969', 'Error rc': 0, 'Time': 0.4620800018310547}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
selected = [(p, [pos for pos in model.position if pe.value(model.x[p,pos])>0.5][0])
            for p in model.player if sum(pe.value(model.x[p,pos]) for pos in model.position)>0.5]
print(f"11 jugadores (jugador, posición): {selected}")

11 jugadores (jugador, posición): [(186153, 'GK'), (178372, 'CD'), (155862, 'CI'), (199914, 'MD'), (146954, 'CM'), (207410, 'MI'), (184949, 'RB'), (192679, 'LB'), (208722, 'RW'), (202166, 'LW'), (9676, 'ST')]


In [68]:
import pandas as pd
import pyomo.environ as pe

def resumen_equipo(model, df, selected, cols_extra=None, ordenar_por="assigned_pos"):
    """
    Devuelve un DataFrame con la info relevante del XI:
    - player_id
    - posición asignada (assigned_pos)
    - nombre, club, liga, nacionalidad
    - overall, value_eur y stats principales (si existen)
    """
    # columnas recomendadas (usa solo las que existan en df)
    base_cols = [
        "player_id", "short_name",
        "club_team_id", "club_name",
        "league_id", "league_name",
        "nationality_id", "nationality_name",
        "position",          # posición original del CSV
        "overall", "value_eur",
        "pace","shooting","passing","dribbling","defending","physic"
    ]
    if cols_extra:
        base_cols += list(cols_extra)

    base_cols = [c for c in base_cols if c in df.columns]

    # mapa player_id -> posición asignada en la solución
    asignada = dict(selected)
    ids = list(asignada.keys())

    # subset + añadir posición asignada
    out = (df[df["player_id"].astype(int).isin(ids)][base_cols]
           .copy())
    out["player_id"] = out["player_id"].astype(int)
    out["assigned_pos"] = out["player_id"].map(asignada)

    # ordenar para que salga como formación
    if ordenar_por == "assigned_pos":
        order = list(model.position)  # respeta el orden de tu Set si es ordered=True
        pos_rank = {p:i for i,p in enumerate(order)}
        out["_rank"] = out["assigned_pos"].map(pos_rank)
        out = out.sort_values("_rank").drop(columns=["_rank"])
    else:
        out = out.sort_values(ordenar_por)

    # (opcional) métricas resumen rápidas
    resumen = {
        "total_value_eur": float(out["value_eur"].sum()) if "value_eur" in out else None,
        "avg_overall": float(out["overall"].mean()) if "overall" in out else None,
    }

    return out.reset_index(drop=True), resumen


# USO:
equipo_df, metrics = resumen_equipo(model, df, selected)
print(equipo_df)
print(metrics)

    player_id    short_name  club_team_id             club_name  league_id  \
0      186153   W. Szczęsny          45.0              Juventus       31.0   
1      178372   B. Ivanović      100769.0  Zenit St. Petersburg       67.0   
2      155862  Sergio Ramos         243.0           Real Madrid       53.0   
3      207410    M. Kovačić         243.0           Real Madrid       53.0   
4      199914         Allan          48.0                Napoli       31.0   
5      146954          Gabi         240.0       Atlético Madrid       53.0   
6      184949       Mariano         325.0           Galatasaray       68.0   
7      192679      Escudero         481.0               Sevilla       53.0   
8      208722       S. Mané           9.0             Liverpool       13.0   
9      202166    J. Draxler          73.0   Paris Saint Germain       16.0   
10       9676      S. Eto'o         741.0           Antalyaspor       68.0   

       league_name  nationality_id nationality_name position  o

In [ ]:
# pares únicos (i<j) a partir del diccionario chemistry_dict (ya simétrico en tu código)
pairs = [(i,j) for (i,j) in chemistry_dict.keys() if i < j]
model.pair = pe.Set(initialize=pairs, dimen=2)

model.y = pe.Var(model.pair, within=pe.Binary)

def y_ub_i(m, i, j):  return m.y[i,j] <= m.sel[i]
def y_ub_j(m, i, j):  return m.y[i,j] <= m.sel[j]
def y_lb(m, i, j):    return m.y[i,j] >= m.sel[i] + m.sel[j] - 1

model.y_ub_i = pe.Constraint(model.pair, rule=y_ub_i)
model.y_ub_j = pe.Constraint(model.pair, rule=y_ub_j)
model.y_lb   = pe.Constraint(model.pair, rule=y_lb)

In [ ]:
##### FUNCION OBJETIVO DE LA QUIMICA
def obj_rule(m):
    return sum(m.chemistry[i,j] * m.y[i,j] for (i,j) in m.pair)
model.obj = pe.Objective(rule=obj_rule, sense=pe.maximize)